In [1]:
using BenchmarkTools
using StaticArrays
using Random
using Plots
using LinearAlgebra

### Benchmarking Code

Julia is just in time (JIT) compiled, so the first time you run code there will be a penalty to first compile the code before it can be run. We can use the `@time` macro to see this effect. (If you run the cell more than once, the effect will go away since it has already been compiled!)

In [2]:
x = rand(1000);
function sum_global()
    s = 0.0
    for i in x
        s += i
    end
    return s
end;

In [3]:
@time sum_global();
@time sum_global();

  0.011895 seconds (3.68 k allocations: 78.109 KiB, 98.53% compilation time)
  0.000158 seconds (3.49 k allocations: 70.156 KiB)


The `@time` macro is convenient, but only executes the function once. We can use the [BenchmarkTools](https://juliaci.github.io/BenchmarkTools.jl/stable/) library to investigate the performance of our function with multiple trials. The `@benchmark` macro will run 10,000 trials or (roughly) 5 seconds whichever comes first. (These can be configured). There is also the `@btime` macro which runs the same number of trials as `@benchmark` but has output similar to `@time`.

In [4]:
@benchmark sum_global()

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   73.742 μs … 202.248 ms  ┊ GC (min … max):  0.00% … 99.94%
 Time  (median):      77.693 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   108.668 μs ±   2.022 ms  ┊ GC (mean ± σ):  19.89% ±  2.97%

  ▄▅█▅▄            ▂▃▄▄▄▂▁▁                                     ▁
  ██████▇▆▅▆▅▆▆▆▆▆▇█████████▇█▇█▇▇▇▆▇▆▆▅▅▆▆▆▅▆▅▅▅▆▄▄▅▅▅▅▃▄▄▃▅▄▄ █
  73.7 μs       Histogram: log(frequency) by time        164 μs <

 Memory estimate: 70.16 KiB, allocs estimate: 3490.

If your function takes parameters, be sure to "interpolate" them with a `$` when using `@benchmark` this tells BenchmarkTools to ignore the allocation and time required to allocate the parameters.

In [5]:
# Parameter is Interpolated
@btime inv($(rand(6,6)));
# No Interpolation
@btime inv(rand(6,6));

  1.733 μs (4 allocations: 3.59 KiB)
  1.951 μs (5 allocations: 3.95 KiB)


### Reducing Memory Allocations
One of the easiest ways to speed up your code is by removing all unnecessary memory allocations. In Python the end-user is not able to manage memory easily and the interprer or library (e.g. numpy) is trusted to handle things.

Julia can also operate like Python, where we just trust the compiler to "do the right thing". This is good for prototyping, but can result in inefficient code. Below we will look at the basics of memory allocations and how to reduce them.

-----------------------------------------------
Lets start by allocating a small array of 10,000 `Float64`s. Each `Float64` is 8 bytes (64 bits), so the total size should be 80,000 bytes or 80kB (78.125 kiB). Note there will be a slight overhead for some internal machinery associated with each allocation (e.g. a pointer to the array).

In [6]:
@time ones(Float64, 100, 100);

  0.000010 seconds (2 allocations: 78.172 KiB)


In Julia all arrays are `heap` allocated since they are re-sizeable. This just means that the array is further away from the CPU and it can be slow(er) to allocate and access. The first strategy to mitigate this penalty is to simply allocate memory as little as possible. To check this we can use BenchmarkTools to understand the allocations in our function.

In [7]:
function allocates_alot()
    res = 0.0
    for i in 1:100
        x = rand(100,100) # ALLOCATION EVERY LOOP ITERATION
        res += sum(x)
    end
    return res
end

function allocates_once()
    res = 0.0
    storage = zeros(100,100)
    for i in 1:100
        # The ! point means it mutates the input
        # In this case it over writes storage with new random values
        rand!(storage)
        res += sum(storage)
    end
    return res
end


allocates_once (generic function with 1 method)

Using the `@btime` macro, we can measure the performace uplift and the number of allocations from each function. You'll find that the function which allocates a new array each loop iteration will be slower and allocate far more memory.

In [8]:
@btime allocates_alot() samples = 1000;
@btime allocates_once() samples = 1000;

  4.800 ms (200 allocations: 7.63 MiB)
  1.052 ms (2 allocations: 78.17 KiB)


A more subtle, yet very common place where allocations occur is array slices (e.g. `arr[1:10]`). In Julia, these slices allocate a new array by default. We can get around this by telling Julia that we only want to read the data inside the arr. To do this we use an "array view" which can be acheived with the `view(...)` function or the `@views` macro.

In [9]:
function slices_allocate(x)
    N = Int(length(x) / 2)
    res = sum(x[1:N])
    return res
end

function view_slices(x)
    N = Int(length(x) / 2)
    #The view function takes the array
    # and the indicies you want a view of.
    res = sum(view(x, 1:N))
    return res
end

function view_macro_slices(x)
    N = Int(length(x) / 2)
    # The @views macro tells Julia all array slices in
    # this line should be array views.
    @views res = sum(x[1:N])
    return res
end

view_macro_slices (generic function with 1 method)

Again we see a large speed increase and the number of allocations drop!

In [10]:
x = rand(100000)
@btime slices_allocate(x) samples = 1000;
@btime view_slices(x) samples = 1000;
@btime view_macro_slices(x) samples = 1000;

  35.536 μs (3 allocations: 390.69 KiB)
  9.059 μs (1 allocation: 16 bytes)
  9.053 μs (1 allocation: 16 bytes)


### StaticArrays
Earlier, we mentioned that Julia arrays are `heap` allocated because they have variable length (i.e., you can append to them). If you know the length of your array beforehand, you can use the [StaticArrays](https://juliaarrays.github.io/StaticArrays.jl/stable/) library to `stack` allocate your data (if its small enough).

StaticArrays has many types you can use, but the main types are `SVector` for 1D arrays, `SMatrix` for 2D arrays, and `SArray` for n-dimensional arrays (all types have equivalent macros e.g., `@SMatrix`).

To initialize a NxN `SMatrix` we must provide the size to the constructor: `SMatrix{N,N}(...)`

In [11]:
m = SMatrix{25,25}(rand(25,25));

A common pattern in atomistic simulation packages is to have an array of `SVectors` which represent atomic properties, like their position. Storing the data for each atom as an SVector can be much faster than using the default Julia array.

In [12]:
N_atoms = 1000
static_positions = [SVector{3}(rand(3)) for i in 1:N_atoms];
dynamic_positions = [rand(3) for i in 1:N_atoms];

function distance_matrix(positions)
    N = length(positions)
    dists = zeros(N,N)
    for i in 1:N
        for j in i:N
            dists[i,j] = norm(positions[i] - positions[j])
            dists[j,i] = dists[i,j]
        end
    end
    return dists
end


distance_matrix (generic function with 1 method)

In [13]:
println("Static Vector Benchmark:")
@benchmark distance_matrix(static_positions)

Static Vector Benchmark:


BenchmarkTools.Trial: 1023 samples with 1 evaluation.
 Range (min … max):  3.331 ms … 20.662 ms  ┊ GC (min … max):  0.00% … 60.81%
 Time  (median):     4.364 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   4.862 ms ±  1.739 ms  ┊ GC (mean ± σ):  13.46% ± 18.04%

  █▆▂▁   ▃█▇▅▃▂▁▁▁             ▂▂▁        ▁▂▂                 
  ████▆▄▁██████████▇▅▆▇▅▆▆▆▄▄▄▅███▇▇▄▄▅▄▁▁██████▆▆▆▆▆▆▅▁▄▄▄▅ █
  3.33 ms      Histogram: log(frequency) by time     10.1 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

In [14]:
println("Dynamic Array Benchmark:")
@benchmark distance_matrix(dynamic_positions)

Dynamic Array Benchmark:


BenchmarkTools.Trial: 104 samples with 1 evaluation.
 Range (min … max):  39.539 ms … 86.751 ms  ┊ GC (min … max): 5.22% … 2.37%
 Time  (median):     49.074 ms              ┊ GC (median):    4.78%
 Time  (mean ± σ):   48.075 ms ±  6.049 ms  ┊ GC (mean ± σ):  5.04% ± 1.62%

  ▁ ▁   ▁  ▁            ▃ ▁ ▁█ ▁▄ ▁ ▃                          
  █▆█▆▆▄█▇▇█▆▁▁▇▁▄▄▄▁▄▆▆█▁█▇██▇██▇█▇█▁▇▄▆▁▁▁▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▄
  39.5 ms         Histogram: frequency by time        61.8 ms <

 Memory estimate: 45.81 MiB, allocs estimate: 500502.

Note that `SVector`s etc. are statically sized and immutable. The StaticArrays package also provides statically sized but mutable variants, which are prefixed with `M` instead of `S`, e.g. `SMatrix`.

### Type Stability
Julia will do its best to infer the data types of your variables, but if the compiler cannot infer the type of all your variables this leads to type instability and will slow down your code. We can check type stability with the `@code_warntype` macro. For today's purposes, if this macro returns any red text, then your code has unstable types.

In the example below, the functions are equivalent except one relies on a gloabl variable and the other takes that variable as a parameter. Because of the global variable, the compiler must assume that the variables inside the *unstable* function have `Any` type. Whereas in the type-stable code, the compiler can create a specialized version of the stable function for the specific type you passed into it. This is not the only unstable pattern, so if you really care about speed, check your code with `@code_warntype`.

In [15]:
global_x = rand(Float64, 1000)
function unstable()
    s = 0
    for val in global_x
      s = s + val
    end
    return s
end

function stable(x)
  s = zero(eltype(x))
  for val in x
     s = s + val
  end
  return s
end

stable (generic function with 1 method)

In [61]:
@code_warntype unstable()

MethodInstance for unstable()
  from unstable() @ Main c:\Users\ejmei\repos\MolSSI_workshop\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X34sZmlsZQ==.jl:2
Arguments
  #self#::Core.Const(unstable)
Locals
  @_2::Any
  s::Any
  val::Any
Body::Any
1 ─       (s = 0)
│   %2  = Main.global_x::Any
│         (@_2 = Base.iterate(%2))
│   %4  = (@_2 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_2::Any
│         (val = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Any
│         (s = s + val)
│         (@_2 = Base.iterate(%2, %9))
│   %12 = (@_2 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return s



In [16]:
@code_warntype stable(global_x)

MethodInstance for stable(::Vector{Float64})
  from stable(x) @ Main In[15]:10
Arguments
  #self#::Core.Const(stable)
  x::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Float64
  val::Float64
Body::Float64
1 ─ %1  = Main.eltype(x)::Core.Const(Float64)
│         (s = Main.zero(%1))
│   %3  = x::Vector{Float64}
│         (@_3 = Base.iterate(%3))
│   %5  = (@_3 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_3::Tuple{Float64, Int64}
│         (val = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│         (s = s + val)
│         (@_3 = Base.iterate(%3, %10))
│   %13 = (@_3 === nothing)::Bool
│   %14 = Base.not_int(%13)::Bool
└──       goto #4 if not %14
3 ─       goto #2
4 ┄       return s



In [17]:
@btime unstable();

  77.671 μs (3490 allocations: 70.16 KiB)


In [18]:
@btime stable($global_x);

  1.128 μs (0 allocations: 0 bytes)
